In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [5]:
!nvidia-smi

Tue Nov 19 00:56:48 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla P100-PCIE-16GB           Off |   00000000:00:04.0 Off |                    0 |
| N/A   38C    P0             27W /  250W |       0MiB /  16384MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

In [7]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
from datasets import load_dataset
import pandas as pd
from datasets import load_dataset

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk
from nltk.tokenize import sent_tokenize

from tqdm import tqdm
import torch

nltk.download("punkt")

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [9]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

In [10]:
# Load the tokenizer for the "google/pegasus-cnn_dailymail" model, used for text summarization tasks.
tokenizer = AutoTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [11]:
# Load the PEGASUS model for sequence-to-sequence tasks, optimized for text summarization, and move it to the specified device (e.g., GPU or CPU).
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained("google/pegasus-cnn_dailymail").to(device)

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [12]:
# Load the SAMSum dataset, which is commonly used for dialogue summarization tasks.
dataset_samsum = load_dataset("samsum")

samsum.py:   0%|          | 0.00/3.36k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

The repository for samsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/samsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


corpus.7z:   0%|          | 0.00/2.94M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14732 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/819 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/818 [00:00<?, ? examples/s]

In [13]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [14]:
dataset_samsum["train"][0]

{'id': '13818513',
 'dialogue': "Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)",
 'summary': 'Amanda baked cookies and will bring Jerry some tomorrow.'}

In [15]:
def convert_examples_to_features(batch):
    """
    Converts a batch of examples from the SAMSum dataset into features suitable for model training.
    """
    # Tokenize the 'dialogue' field with padding to the maximum length and truncation.
    input_encodings = tokenizer(batch['dialogue'], padding="max_length", truncation=True)

    # Tokenize the 'summary' field with a maximum length of 128 and truncation.
    target_encodings = tokenizer(batch['summary'], max_length=128, truncation=True)

    return {
        'input_ids': input_encodings['input_ids'],       # Token IDs for the input dialogues
        'attention_mask': input_encodings['attention_mask'],  # Attention mask for the input dialogues
        'labels': target_encodings['input_ids']         # Token IDs for the target summaries
    }

In [16]:
# Apply the `convert_examples_to_features` function to the SAMSum dataset to tokenize the dialogues and summaries, 
# transforming the dataset into a format suitable for model training. The 'batched=True' argument allows the function 
# to process multiple examples at once for efficiency.
dataset_samsum = dataset_samsum.map(convert_examples_to_features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [17]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 818
    })
})

In [18]:
from transformers import DataCollatorForSeq2Seq

# Create a data collator for sequence-to-sequence tasks that handles dynamic padding and prepares batches for the model.
# It uses the tokenizer and model (PEGASUS) to properly format the input sequences and labels during training.
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

In [19]:
from transformers import TrainingArguments, Trainer

# Set up the training arguments for the Trainer, specifying various hyperparameters and training configurations.
# This includes the output directory, number of epochs, batch sizes, weight decay, and logging/evaluation settings.
trainer_args = TrainingArguments(
    output_dir='/kaggle/working/',          # Directory where model checkpoints and logs will be saved
    num_train_epochs=1,                  # Number of training epochs
    warmup_steps=500,                    # Number of steps for learning rate warmup
    per_device_train_batch_size=1,       # Batch size for training
    per_device_eval_batch_size=1,        # Batch size for evaluation
    weight_decay=0.01,                   # Weight decay to avoid overfitting
    logging_steps=10,                    # Frequency of logging steps
    evaluation_strategy='steps',         # Evaluate the model at each evaluation step
    eval_steps=500,                      # Frequency of evaluation steps
    save_steps=1e6,                      # Save model checkpoint after every 1 million steps
    gradient_accumulation_steps=16       # Number of steps to accumulate gradients before performing a backward pass
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [20]:
# Initialize the Trainer with the model, training arguments, tokenizer, data collator, and datasets for training and evaluation.
# This sets up the environment for training and evaluating the PEGASUS model on the SAMSum dataset.
trainer = Trainer(
    model=model_pegasus,                     # The PEGASUS model to be trained
    args=trainer_args,                       # The training arguments specifying hyperparameters and settings
    tokenizer=tokenizer,                     # The tokenizer to process text data
    data_collator=seq2seq_data_collator,     # Data collator for padding and formatting batches
    train_dataset=dataset_samsum["train"], # The test dataset to use for training
    eval_dataset=dataset_samsum["test"]  # The validation dataset to use for evaluation during training
)

In [21]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss,Validation Loss
500,1.658000,1.511652


/opt/conda/lib/python3.10/site-packages/transformers/modeling_utils.py:2618: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=920, training_loss=1.8314198929330576, metrics={'train_runtime': 5734.8577, 'train_samples_per_second': 2.569, 'train_steps_per_second': 0.16, 'total_flos': 4.253291617714176e+16, 'train_loss': 1.8314198929330576, 'epoch': 0.9991854466467553})

In [22]:
## Save model
model_pegasus.save_pretrained("/kaggle/working/")
## Save tokenizer
tokenizer.save_pretrained("/kaggle/working/")

('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/spiece.model',
 '/kaggle/working/added_tokens.json',
 '/kaggle/working/tokenizer.json')

In [38]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the fine-tuned model and tokenizer from the saved checkpoint
model_ckpt = "/kaggle/working/"  # Path where the model was saved after fine-tuning
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)  # Load the tokenizer, which converts text to token IDs
model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)  # Load the fine-tuned model and move it to the specified device (GPU or CPU)

# Function to generate a summary for the input dialogue text
def generate_summary(input_text):
    """
    Takes input_text (dialogue) as input and generates a summary using the pre-trained Pegasus model.
    It tokenizes the input text, feeds it into the model, and decodes the output tokens into a summary.
    """

    # Tokenize the input text and prepare it for the model
    inputs = tokenizer(input_text, 
                       return_tensors="pt",  # Return tokenized inputs as PyTorch tensors, which is the format the model expects
                       padding="max_length",  # Pad input text to the maximum length (1024 tokens). Ensures uniform input size across batches.
                       truncation=True,  # Truncate input if it exceeds max_length to fit the model's input size.
                      ).to(device)  # Move tensors to the correct device (GPU or CPU).
    
    # Generate the summary from the model
    summary_ids = model_pegasus.generate(
        inputs['input_ids'],  # The tokenized input text is passed to the model
        num_beams=4,  # Use beam search with 4 beams, which generates multiple sequences and selects the best one. Helps in improving summary quality.
        max_length=128,  # The length of the generated summary. The summary will be capped at 128 tokens.
        early_stopping=True)  # Stop the generation process as soon as the model generates an end-of-sequence token (avoids generating unnecessary tokens)
    
    # Decode the tokenized summary back into readable text
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)  # Converts the generated token IDs back to human-readable text, skipping special tokens like <eos>, <pad>, etc.
    
    return summary  # Return the generated summary

# Example test dialogue to test the model's summarization capabilities
test_dialogue = dataset_samsum["train"][0]["dialogue"]

# Call the generate_summary function and print the result
summary = generate_summary(test_dialogue)
print("Generated Summary:", summary)  # Output the generated summary of the input dialogue

Generated Summary: Amanda baked cookies. She will bring them tomorrow. Jerry will buy them. Amanda will bring them to him. Jerry will bring them to Amanda. Amanda will bring them tomorrow.


In [37]:
dataset_samsum["train"][0]["dialogue"]

"Amanda: I baked  cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"